In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA

In [3]:
df = pd.read_json('https://www.dropbox.com/s/r7r92hprnx6pfl3/joinTablas.json?dl=1')

In [6]:
display(df.head(3))

,RowID,Fecha,IDItem,NumeroUbicacion,Dewey,Ubicacion,Llave,Programa,Facultad,IDUsuario,Year,Signatura,TipoItem,Autor,Titulo,AnioPublicacion,DeweyEspecifico,TemaDewey,Temas,Union
0,Row0,1547050497000,80000005327627,720.98 A71S,720,COL-GRAL-2,866245,CARRERA DE ARQUITECTURA,arquitectura y diseño,c361c772cd0220bb16dcfb2ad803e4903334ab94,2019,720,LIBRO,"Arango Cardinal, Silvia 1948-",Ciudad y arquitectura seis generaciones que co...,2012.0,720.98,Arquitectura latinoamericana,"[arquitectur, histori, amer, latin]",arquitectur histori amer latin
1,Row1,1547050500000,80000001191496,720.9861 A71,720,COL-GRAL-2,309945,CARRERA DE ARQUITECTURA,arquitectura y diseño,c361c772cd0220bb16dcfb2ad803e4903334ab94,2019,720,LIBRO,"Arango Cardinal, Silvia 1948-",Historia de la arquitectura en Colombia Silvia...,1993.0,720.9861,Arquitectura colombiana,"[arquitectur, colonial, histori, colombi]",arquitectur colonial histori colombi
2,Row2,1547138763000,80000004979759,540 CH15Q 2010,540,COL-GRAL-3,822727,CARRERA DE ARQUITECTURA,arquitectura y diseño,87b0e5a61ed712ddfaf5d478ad68c87c825997e9,2019,540,LIBRO,"Chang, Raymond",Química Raymond Chang ; revisión técnica Rosa ...,2010.0,540,Química,[quimic],quimic


In [7]:
display(df.columns)

Index(['RowID', 'Fecha', 'IDItem', 'NumeroUbicacion', 'Dewey', 'Ubicacion',
       'Llave', 'Programa', 'Facultad', 'IDUsuario', 'Year', 'Signatura',
       'TipoItem', 'Autor', 'Titulo', 'AnioPublicacion', 'DeweyEspecifico',
       'TemaDewey', 'Temas', 'Union'],
      dtype='object')

In [8]:
df["Peso"] = df.apply(lambda row: 1/2**(2021-row.Year), axis=1 )

In [9]:
pesos_usuarios = pd.DataFrame(columns= df["Signatura"].unique(),dtype=float)
tam = len(df["Signatura"].unique())
pesos_usuarios.columns = pesos_usuarios.columns.astype(str)
display(pesos_usuarios)

ids = df["IDUsuario"].unique()
i = 0
for usuario in ids:
  pesos_usuarios.loc[i] = np.zeros((tam,), dtype=int)
  #Para cada usuario traer sus Deweys y sus pesas asociados
  prestamos = df.loc[df["IDUsuario"]==usuario]
  deweys = prestamos[["Signatura","Peso"]]
  result_user = deweys.groupby("Signatura")["Peso"].sum().reset_index(name="Peso")
  # Recorrer los Deweys y Pesos para plasmarlos en una matriz
  #display(result_user)
  for index, row in result_user.iterrows():
    d = row["Signatura"]
    p=row['Peso']
    pesos_usuarios[d][i] = p
  i = i+1

,720,540,512,712,741,814,823,330,863,696,...,267,114,034,673,447,600,653,263,091,586


In [10]:
pesos_usuarios.to_json(r'C:\Users\johan\Desktop\GitKraken\DataOutOfGit\Limpieza\pesos_usuario_x_dewey.json')

In [11]:
sumatoria = pesos_usuarios.sum(axis=1)
pesos_norm = pesos_usuarios.div(sumatoria, axis=0).fillna(0)

In [13]:
print(type(pesos_norm))

<class 'pandas.core.frame.DataFrame'>


In [22]:
pesos_norm_id = pesos_norm.copy()
pesos_norm_id['IDUsuario'] = df.IDUsuario.unique()
display(pesos_norm_id.shape[0])

24407

In [33]:
def get_temas_usuario(id_usuario, num_deweys):
    pesos_usuario = pesos_norm_id.loc[pesos_norm_id.IDUsuario == id_usuario]
    display(pesos_usuario)
    deweys_importantes = []
    df_temp = pesos_usuario.copy()

    for i in range(0, num_deweys):
        maxvalueIndexLabel = df_temp.idxmax(axis = 1)
        deweys_importantes.append(maxvalueIndexLabel.values[0])
        df_temp = df_temp.drop(labels = maxvalueIndexLabel.values, axis = 1)

    df_cluster_interes = df_cluster_interes.filter(items=deweys_importantes)
    display(df_cluster_interes)

In [34]:
get_temas_usuario('c361c772cd0220bb16dcfb2ad803e4903334ab94', 15)

,720,540,512,712,741,814,823,330,863,696,...,114,034,673,447,600,653,263,091,586,IDUsuario
0,0.380952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02381,c361c772cd0220bb16dcfb2ad803e4903334ab94


TypeError: reduction operation 'argmax' not allowed for this dtype

In [39]:
def eliminar_cero(id_usuario):
    m1 = (pesos_norm_id['IDUsuario'] == id_usuario)
    m2 = (pesos_norm_id[m1] != 0).all()
    return pesos_norm_id.loc[m1,m2]

In [41]:
display(eliminar_cero('c361c772cd0220bb16dcfb2ad803e4903334ab94'))

,720,711,725,726,428,746,155,770,378,589,771,584,582,586,IDUsuario
0,0.380952,0.047619,0.02381,0.095238,0.095238,0.047619,0.047619,0.047619,0.095238,0.02381,0.02381,0.02381,0.02381,0.02381,c361c772cd0220bb16dcfb2ad803e4903334ab94


In [66]:
def clustering(df_pesos, treshold):
    i = 0
    resultados = pd.DataFrame(columns = ['IDUsuario', 'Cluster', 'Pertenencia'])
    for usuario in df_pesos.IDUsuario.unique():
        usuario_limpio = eliminar_cero(usuario)
        usuario_limpio = usuario_limpio.drop(labels = 'IDUsuario', axis = 1)
        for column in usuario_limpio:
            if usuario_limpio.iloc[0][column] >= treshold:
                resultados.loc[i] = np.array([usuario, column, usuario_limpio.iloc[0][column]])
                i = i + 1
    return resultados    

In [68]:
pesos_clustering = clustering(pesos_norm_id, 0)
display(pesos_clustering)

KeyboardInterrupt: 

In [69]:
pesos_clustering.to_json(r'C:\Users\johan\Desktop\GitKraken\DataOutOfGit\Limpieza\pesos_clustering.json')